## Day 20: Particle Swarm

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

### Part one

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the `X`, `Y`, and `Z` coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format `<X,Y,Z>`.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:

 - Increase the X velocity by the X acceleration.
 - Increase the Y velocity by the Y acceleration.
 - Increase the Z velocity by the Z acceleration.
 - Increase the X position by the X velocity.
 - Increase the Y position by the Y velocity.
 - Increase the Z position by the Z velocity.

Because of seemingly tenuous rationale involving [z-buffering](https://en.wikipedia.org/wiki/Z-buffering), the GPU would like to know which particle will stay closest to position <0,0,0> in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:

p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   

> Let's begin with a parser to get all the variables

In [1]:
import re
import numpy as np

pat_particle = re.compile('<([ -]?\d+),(-?\d+),(-?\d+)>')
lines = [
    'p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>',
    'p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>',
]
pos = []
vel = []
acc = []
for l in lines:
    p, v, a = pat_particle.findall(l)
    p = tuple(( int(_) for _ in p))
    pos.append(p)
    v = tuple(( int(_) for _ in v))
    vel.append(v)
    a = tuple(( int(_) for _ in a))
    acc.append(a)

pos = np.array(pos)
vel = np.array(vel)
acc = np.array(acc)


for _ in range(3):
    vel += acc
    pos += vel
    
pos

array([[ 3,  0,  0],
       [-8,  0,  0]])

At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position <0,0,0> in the long term?

In [2]:
np.sign(pos), np.sign(vel)

(array([[ 1,  0,  0],
        [-1,  0,  0]]), array([[-1,  0,  0],
        [-1,  0,  0]]))

In [3]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y', 'z'])
zero = Point(0,0,0)
assert zero.x == 0
assert zero.y == 0
assert zero.z == 0

In [4]:
def load_input(filename):
    pat_particle = re.compile('<([ -]?\d+),(-?\d+),(-?\d+)>')
    with open(filename, 'r') as f:
        lines = [_.strip() for _ in f.readlines()]       
    pos = []
    vel = []
    acc = []
    for l in lines:
        p, v, a = pat_particle.findall(l)
        pos.append( [ int(_) for _ in p] )
        vel.append( [ int(_) for _ in v] )
        acc.append( [ int(_) for _ in a] )
    return np.array(pos), np.array(vel), np.array(acc)

In [5]:
pos, vel, acc = load_input('input_test.txt')
assert np.all(pos == np.array([[3, 0, 0], [4, 0, 0]]))
assert np.all(vel == np.array([[2, 0, 0], [0, 0, 0]]))
assert np.all(acc == np.array([[-1, 0, 0], [-2, 0, 0]]))

In [6]:
for i in range(3):
    vel += acc
    pos += vel
print(pos[0], pos[1])

[3 0 0] [-8  0  0]


In [7]:
def acc_equals_vel(v, a): # acc y vel on the same direction, or acc is zero
    t = np.logical_or( np.sign(acc) == np.sign(vel), acc == np.zeros(acc.shape) )
    return np.all(t)

In [8]:
acc_equals_vel(vel, acc)

True

In [9]:
pos, vel, acc = load_input('input_test.txt')
while True:
    if acc_equals_vel(vel, acc):
        break
    vel += acc
    pos += vel
pos, np.min(pos)

(array([[ 3,  0,  0],
        [-8,  0,  0]]), -8)

In [10]:
def manhattan_distance(v):
    x = np.abs(v[:, 0])
    y = np.abs(v[:, 1])
    z = np.abs(v[:, 2])
    return x + y + z

In [11]:
print(np.argmin(manhattan_distance(pos)))

0


In [12]:
manhattan_distance(pos)

array([3, 8])

In [13]:
pos, vel, acc = load_input('input.txt')
counter = 0
while True:
    counter += 1
    if counter % 1000 == 0:
        print('#', end='')
    if counter == 5000000:
        break
    if acc_equals_vel(vel, acc):
        break
    vel += acc
    pos += vel
# let's do some iterations more, just to be sure
for i in range(100000):
    vel += acc
    pos += vel
index = np.argmin(manhattan_distance(pos))
print('Part one: {}'.format(index))


Part one: 258


In [14]:
pos, vel, acc = load_input('input.txt')

### Part two

To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:

    p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
    p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
    p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
    p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

    p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
    p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
    p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
    p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

    p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
    p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
    p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
    p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

    ------destroyed by collision------    
    ------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
    ------destroyed by collision------                      (3)         
    p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>

In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

**How many particles are left after all collisions are resolved?**

> Function `np.unique` returns the sorted unique elements of an array. There are three optional outputs in addition to the unique elements: the indices of the input array that give the unique values, the indices of the unique array that reconstruct the input array, and the number of times each unique value comes up in the input array.
Parameters:	

> - `return_index` : bool, optional. If True, also return the indices of array (along the specified axis, if provided, or in the flattened array) that result in the unique array.

> - `return_counts` : bool, optional. If True, also return the number of times each unique item appears in ar. .. versionadded:: 1.9.0

> - `axis` : int or None, optional. The axis to operate on. If None, ar will be flattened beforehand. Otherwise, duplicate items will be removed along the provided axis, with all the other axes belonging to the each of the unique elements. Object arrays or structured arrays that contain objects are not supported if the axis kwarg is used. .. versionadded:: 1.13.0


In [15]:
pos, vel, acc = load_input('input_test_part_2.txt')
for i in range(2):
    vel += acc
    pos += vel
    pos, ii, cc = np.unique(pos, return_index=True, return_counts=True, axis=0)
    acc = acc[ii]
    vel = vel[ii]
    pos = pos[cc == 1]
    acc = acc[cc == 1]
    vel = vel[cc == 1]
print('pos:', pos)
print('vel:', vel)

pos: [[1 0 0]]
vel: [[-1  0  0]]


In [16]:
pos, vel, acc = load_input('input.txt')
for i in range(40000):
    vel += acc
    pos += vel
    pos, ii, cc = np.unique(pos, return_index=True, return_counts=True, axis=0)
    acc = acc[ii]
    vel = vel[ii]
    pos = pos[cc == 1]
    acc = acc[cc == 1]
    vel = vel[cc == 1]
print('Part two:', len(pos))

Part two: 707
